In [ ]:
%matplotlib nbagg
# import seaborn as sns
# sns.set_context('notebook')

In [ ]:
from planet4 import io, clustering, plotting, markings

In [ ]:
def cluster_and_plot(image_id, eps=10, dynamic=False, angles=False, 
                     scaler='scale', delta_fan_angle=10, delta_blotch_angle=10,
                     dir_ext='_check1', msf=0.15, data=None):
    from planet4 import plotting, clustering
    cm = clustering.ClusteringManager(do_dynamic_min_samples=dynamic,
                                      include_angle=angles,
                                      quiet=False,
                                      output_dir='old_DBSCAN'+dir_ext,
                                      min_samples_factor=msf,
                                      use_DBSCAN=True,
                                      eps=eps,
                                      eps_fanangle=delta_fan_angle,
                                      eps_blotchangle=delta_blotch_angle)

    cm.cluster_image_id(image_id, data=data)
    plotting.plot_image_id_pipeline(image_id,
                                    save=True, 
                                    savetitle='dbscan_',
                                    cm=cm,
                                    with_center=True
                                   )
#     plt.close('all')
    return dict(id_=image_id)

In [ ]:
def create_and_save_randoms():
    myids = np.random.choice(ids, 100)
    np.save('myids.npy', myids)

In [ ]:
io.data_root

In [ ]:
myids = np.load('myids.npy')

len(myids)

In [ ]:
def do_parallel(func, list_):
    from ipyparallel import Client
    from nbtools import display_multi_progress
    c = Client()
    lbview = c.load_balanced_view()
    results = lbview.map_async(func, list_)
    display_multi_progress(results, list_)

In [ ]:
blotchcols = markings.Blotch.to_average
blotchcols
fancols = markings.Fan.to_average
fancols

In [ ]:
def normalize_blotch_angle(df):
    data = df.copy()
    r1_lt_r2 = data.radius_1 < data.radius_2
    blotches = data.marking=='blotch'
    col_orig = ['radius_1','radius_2']
    col_reversed = list(reversed(col_orig))
    data.loc[r1_lt_r2&blotches, col_orig] = data.loc[r1_lt_r2&blotches, col_reversed].values
    data.loc[r1_lt_r2&blotches, 'angle'] += 90
    data.loc[blotches, 'angle'] = data.loc[blotches, 'angle'] % 180
    return data 

In [ ]:
imgid = '1at'
imgid = 'dch'
imgid = 'bvc'
imgid = '1dr'
imgid = '1fe'
p4id = markings.ImageID(imgid, scope='planet4')

In [ ]:
%time cluster_and_plot(imgid, eps=20, delta_blotch_angle=30)

In [ ]:
data = p4id.get_blotches()

In [ ]:
from planet4.dbscan import DBScanner

In [ ]:
current_X = data[['x','y']].values

In [ ]:
clusterer = DBScanner(current_X, eps=10, min_samples=3)

In [ ]:
clusterer.n_clusters_

In [ ]:
cluster = data.loc[clusterer.clustered_indices[8]]

p4id.plot_blotches(blotches=cluster,with_center=True)

In [ ]:
cluster[markings.Blotch.to_average]

In [ ]:
from sklearn.cluster import DBSCAN

class DBScanner(object):

    """Execute clustering and create mean cluster markings.

    The instantiated object will execute:

        * _run_DBSCAN() to perform the clustering itself
        * _post_analysis() to create mean markings from the clustering results


    Parameters
    ----------
    current_X : numpy.array
        array holding the data to be clustered, preprocessed in ClusterManager
    eps : int, optional
        Distance criterion for DBSCAN algorithm. Samples further away than this value don't
        become members of the currently considered cluster. Default: 10
    min_samples : int, optional
        Mininum number of samples required for a cluster to be created. Default: 3
    """

    def __init__(self, X, eps=10, min_samples=3):
        self.X = X
        self.eps = eps
        self.min_samples = min_samples

        # these lines execute the clustering
        self._run_DBSCAN()

    def _run_DBSCAN(self):
        """Perform the DBSCAN clustering."""
        db = DBSCAN(self.eps, self.min_samples).fit(self.X)
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True

        labels = db.labels_
        unique_labels = set(labels)
        colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
        
        self.n_clusters_ = len(unique_labels) - (1 if -1 in labels else 0)

        self.clustered_indices = []  # list of `kind` cluster average objects
        self.n_rejected = 0
        # loop over unique labels.
        for k, col in zip(unique_labels, colors):
            # get indices for members of this cluster
            class_member_mask = (labels == k)
            cluster_members = (class_member_mask & core_samples_mask)

            if k == -1:
                col = 'k'
                self.n_rejected = len(cluster_members)
            else:
                xy = self.X[cluster_members]
                plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
                         markeredgecolor='k', markersize=14)
                xy = self.X[class_member_mask & ~core_samples_mask]
                plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
                         markeredgecolor='k', markersize=6)
                self.clustered_indices.append(cluster_members)
        plt.gca().invert_yaxis()
        plt.title('Estimated number of clusters: %d' % self.n_clusters_)
        self.db = db



In [ ]:
plt.figure(figsize=(5*1.3,5))
clusterer = DBScanner(data[['x','y']].values)

In [ ]:
cluster.user_name.value_counts()

In [ ]:
def cluster_angles(filtered, delta=20):
    # calculated value of euclidean distance of unit vector end points per degree
    dist_per_degree = 0.017453070996747883
    X = angle_to_xy(filtered.angle)
    clusterer = DBScanner(X, eps=delta * dist_per_degree,
                          min_samples=self.min_samples)
    return clusterer.clustered_indices


In [ ]:
from planet4.clustering import angle_to_xy

In [ ]:
filtered = cluster.groupby('user_name').first()

In [ ]:
plt.figure()
filtered.angle.hist()

In [ ]:
angle_to_xy(cluster.angle)

In [ ]:
cluster.shape

In [ ]:
toprint = cluster2[markings.Fan.to_average + ['user_name', 'marking', 'classification_id']]

In [ ]:
toprint.to_clipboard(index=False)

In [ ]:
def add_angle_vector(df):
    new = df.copy()
    new['xang'] = np.cos(np.deg2rad(df.angle))
    new['yang'] = np.sin(np.deg2rad(df.angle))
    return new

In [ ]:
cluster2 = add_angle_vector(cluster2)

In [ ]:
cluster2


# testing angle deltas

In [ ]:
def angle_to_xy(angle):
    x = np.cos(np.deg2rad(angle))
    y = np.sin(np.deg2rad(angle))
    return np.vstack([x,y]).T

In [ ]:
def cluster_angles(angles, delta_angle):
    dist_per_degree = 0.017453070996747883
    X = angle_to_xy(angles)
    clusterer = DBScanner(X, eps=delta_angle*dist_per_degree, min_samples=3)
    return clusterer

In [ ]:
clusterer = cluster_angles(cluster.angle, 10)

In [ ]:
clusterer.db.core_sample_indices_

In [ ]:
clusterer.db.labels_

In [ ]:
cluster.shape

In [ ]:
clusterer.clustered_indices

In [ ]:
cluster2.iloc[clusterer.clustered_data[0]]

In [ ]:
dbscanner.reduced_data[0]

this means all ellipses were clustered together. eps=10 picks 3 out of these 6.

In [ ]:
clusterdata = data.iloc[dbscanner.reduced_data[0]]

so clusterdata is just the same as the input data, i just repeat the exact same code steps here for consistency.

In [ ]:
clusterdata[blotchcols]

In [ ]:
meandata = clusterdata.mean()
meandata

In [ ]:
from scipy.stats import circmean

In [ ]:
meandata.angle = circmean(clusterdata.angle, high=180)

In [ ]:
meandata

In [ ]:
from sklearn.preprocessing import StandardScaler, robust_scale

scaler = StandardScaler().fit(X)

tX = robust_scale(X)

In [ ]:
import hdbscan
def do_hdbscan(min_cluster, min_samples=None):
    clusterer = hdbscan.HDBSCAN(min_cluster_size=min_cluster, 
                                approx_min_span_tree=False,
                                min_samples=min_samples,
                               )

    imgid.plot_fans()

    db = clusterer.fit(tX)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.probabilities_ > 0.75] = True
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

    unique_labels = set(labels)
    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = 'k'

        class_member_mask = (labels == k)

        xy = X[class_member_mask & core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
                 markeredgecolor='k', markersize=14)

        xy = X[class_member_mask & ~core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
                 markeredgecolor='k', markersize=6)

    plt.title('Estimated number of clusters: %d' % n_clusters_)
    
    reduced_data = []
    for k in unique_labels:
        class_member_mask = (labels == k)
        if k == -1:
            continue
        else:
            reduced_data.append(class_member_mask & core_samples_mask)
    return reduced_data

In [ ]:
reduced_data = do_hdbscan(4, 1)

In [ ]:
blotches[reduced_data[0]]['x y'.split()]

In [ ]:
tX.shape

In [ ]:
plt.figure()
db.condensed_tree_.plot(select_clusters=True)

In [ ]:
cm.dbname

In [ ]:
db = io.DBManager()

In [ ]:
data = db.get_image_id_markings('bvc')

In [ ]:
n_class_old = data.classification_id.nunique()
n_class_old

In [ ]:
# number of classifications that include fan and blotches
f1 = data.marking == 'fan'
f2 = data.marking == 'blotch'
n_class_fb = data[f1 | f2].classification_id.nunique()
n_class_fb

In [ ]:
data=data[data.marking=='blotch']

In [ ]:
plotting.plot_raw_blotches('bvc')

In [ ]:
fans.plot(kind='scatter', x='x',y='y')
plt.gca().invert_yaxis()

In [ ]:
fx1 = data.x < 400 
fx2 = data.x > 300
fy1 = data.y_R > 300
fy2 = data.y_R < 400

In [ ]:
data = data.reset_index()

In [ ]:
data[fx1 & fx2 & fy1 & fy2].angle

In [ ]:
cm.dbscanner.reduced_data